## 1.2 从任意长度的可迭代对象中分解元素

### 去掉首尾成绩 (快速得到中间元素) 利用  **\*表达式**，分解未知或任意长度的可迭代对象

In [1]:
from numpy import mean

In [2]:
st_scores =[1,2,3,4,5,6]
# a,b... = st_scores 这种形式或者循环或得值太麻烦

In [5]:
def drop_first_last(grades):
    firs, *middle, last =grades # *var 代表中间中间变量的列表
    return mean(middle)
drop_first_last(st_scores)

3.5

In [2]:
#分解后丢弃某些不需要的值, *_ 表示带丢弃的变量名
record =('ACM', 50 ,1234.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name)
year

ACM


2012

## 1.3 保存最后N个元素

##### 保存有限的历史记录算是collections.[deque](https://blog.csdn.net/qq_39478403/article/details/105828125)的完美应用场景 ; (类似 list 的容器，两端都能实现快速 append 和 pop (双端队列))

In [3]:
from collections import deque

In [7]:
q = deque(maxlen=2) #deque(maxlen=N)创建了一个固定长度的双向队列。当有新记录加入而队列已满时会自动移除最老的记录
q.append(1) ; q.append(2) ;q.append(3) 
q

deque([2, 3])

在队列两端执行添加和弹出,复杂度O(1);列表的头部插入或移除复杂度O(N)

In [9]:
q.appendleft(1)

In [10]:
q.popleft() #pop为右侧弹出

1

#### 对文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本
 注：包含[yield](https://blog.csdn.net/mieleizhi0522/article/details/82142856/)关键字，就变成了生成器函数，返回的是一个迭代器 (类似return) 

In [11]:
def search(lines,pattern,history=5):
    previous_lines = deque (maxlen=history)
    for line in lines: 
        if pattern in line: #当前行中存在匹配字符串pattern
            yield line, previous_lines
        previous_lines.append(line)  #当前行

In [22]:
def search_test():
    with open ('pline.txt')as f: 
        for line,prevlines in search(f,'python',5): #得到search函数返回的迭代器
            for pline in prevlines:
                print(pline, end='')
            print(line, end='') #默认换行，end修改
            print('**'* 4)

In [23]:
search_test()

this is test
this is test
this is test
this is python test 
********
this is test
this is test
this is test
this is python test 
this is python test 
********


## 1.4 找到最大或最小的N个元素
#### heap的**nlargest** **nsmallest** 


In [32]:
import heapq  #堆队列算法

In [44]:
nums1=[1,8,2,23,7,-4,18,23,42,37,2]
print(heapq.nlargest(3,nums1))  #加key参数可操作更复杂的数据结构如字典等。。。
print(heapq.nsmallest(3,nums1))

[42, 37, 23]
[-4, 1, 2]


#### 如果正在寻找最大或最小的N个元素，且同集合中元素的总数目相比，N很小
`heapq.heapify(list)` 将元素以堆的顺序排列,堆最重要的特性就是list[0]总是最小那个的元素。
此外，接下来的元素可依次通过`heapq.heappop()`方法轻松找到。该方法会将第一个元素（最小的）弹出，**然后以第二小的元素取而代之**（这个操作的复杂度是O(1ogN),N代表堆的大小）

注：**列表赋值，也就是浅拷贝**，属于引用类型，被赋值的列表与源列表共同引用一个地址  
新列表的值发生改变，源列表的值也跟着变，若list_heap1 = nums1 ，则lh变得时候，nu也会改变

In [45]:
list_heap1 = list(nums1)  
heapq.heapify(list_heap1)
list_heap1 # 列表堆(小)形式满足 a[k] <= a[2*k+1] and a[k] <= a[2*k+2]

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [41]:
heapq.heappop(list_heap1) 

-4

当所要找的元素数量相对较小时，函数nlargest(0和nsmallestO)才是最适用的  
如果N和集合本身的大小差不多大，通常更快的方法是先对集合排序，然后做切片操作

## 1.5 实现优先级队列

#### 我们想要实现一个队列，它能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。

In [62]:
class PriorityQueue: 
    def __init__(self) -> None:
        self._queue = []  #保证属性不重复，可以在其前面加上单个下划线
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))  #将一个元组(-priority,index,item)作为整体添加到队列。大顶堆
        self._index += 1  #priority相同时比较index
    
    def pop(self):
        return heapq.heappop(self.queue)[-1] #返回元素为  按-priority和index排序后第一个元素(上述元组)的最后一个元素(item)

In [57]:
#演示如何使用上述类
class Item: 
    def __init__ (self, name):
         self.name = name 
    def __repr__ (self): 
        return 'Item ({!r})'.format (self.name)

In [60]:
pq = PriorityQueue()

In [61]:
pq.push(Item('foo'), 1)

上面的代码片段的核心在于heapq模块的使用。函数heapq.heappush()以及heapq.heappop(分别实现将元素从列表queue中插入和移除，且保证列表中第一个元素的优先级最低（如1.4节所述)。  
heappop方法总是返回“最小”的元素，因此这就是让队列能弹出正确元素的关键。此外，由于push和pop操作的复杂度都是O(logN), 其中N代表堆中元素的数量，因此就算N的值很大，这些操作的效率也非常高。  
把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排列。  这和正常的堆排列顺序相反，一般情况下堆是按从小到大的顺序排序的。
变量index的作用是为了将具有相同优先级的元素以适当的顺序排列。  
通过维护一个不断递增的索引，元素将以它们入队列时的顺序来排列。但是，index在对具有相同优先级的元素间做比较操作时同样扮演了重要的角色。

## 1.6 在字典中将键映射到多个值上

## 1.6 在字典中将键映射到多个值上

#### 需要一个能将key映射到多个值得字典（即所谓的一键多值字典）
解决方案：字典每个键都映射到一个单独的值上。如果想映射到多个值，**需要将多个值保存到另一个容器** 如列表或集合


In [2]:
mvd = { #多值字典
    'a' : [1,2,3],
    'b' : (4,5) #希望消除重复元素且无所谓顺序，用集合
}

为了能方便地创建这样的子典，可以利用collections模块中的defaultdict类。  
defaultdict 的一个特点就是它会自动初始化第一个值，这样只需关注添加元素即可

In [4]:
from collections import defaultdict
dl = defaultdict(list)  # 集合设为set
dl['a'].append(1); dl['a'].append(2)
dl['b'].append(11)

原则上，构建一个一键多值字典是很容易的  
但是如果试着自己对第一个值做初始化操作，这就会变得很杂乱  
例如，可能会写下这样的代码：
```python
d={} 
for key,value in pairs: 
    if key not in d: 
        d[key] = [] #key不存在，先创建空列表值，再添加
    d[key].append(value)
```
用defaultdict后代码会清晰很多
```python
d = defaultdict(list) 
    for key,value in pairs: 
        d[key].append(value)
```

## 1.7 让字典保持有序
#### 1.7.1  问题：我们想创建一个字典，同时当对字典做迭代或序列化操作时，也能控制其中元素的顺序
#### 解决：collection模块的**OrderdDict**,迭代时，它会严格按照元素初始添加的顺序进行

In [6]:
from collections import OrderedDict 

In [ ]:
od = OrderedDict()
od['foo']=1 ; od['bar']=2 ; od['spam']=3
for key in od: print(key)

foo
bar
spam


当想构建一个映射结构以便稍后对其做序列化或编码成另一种格式时
OrderedDict就显得特别有用。  
例如，如果想在进行**JSON编码时精确控制各字段的顺序**，那么只要首先在OrderedDict中构建数据就可以了。

In [8]:
import json 
json.dumps(od) 

'{"foo": 1, "bar": 2, "spam": 3}'

OrderedDict内部维护了一个双向链表，它会根据元素加入的顺序来排列键的位置。第一个新加入的元素被放置在链表的末尾。接下来对已存在的键做重新赋值不会改变键的顺序。  
```请注意OrderedDict的大小是普通字典的2倍多，这是由于它额外创建的链表所致。```因此，如果打算构建一个涉及大量OrderedDict实例的数据结构（例如从CSV文件中读取l00000行内容到OrderedDict列表中)，那么需要认真对应用做需求分析，从而判断带来好处是否能超越因额外开销带来的内存缺点

## 1.8 与字典有关的计算
#### 问题：想在字典上对数据执行各样计算（最值、排序等）

In [10]:
sp ={ 'ACME':45.23, 'AAPL':612.78, 'IBM':205.55, 'HPQ':37.20, 'FB':10.75} #股票价格

为了能**对字典内容**做些有用的**计算**，通常会利用**zip()将字典的键和值反转过来** 例如， 下面的代码会告诉我们如何找出价格最低和最高的股；排序利用zip与sorted配合


In [15]:
min_sp = min(zip(sp.values(), sp.keys()))
max_sp = max(zip(sp.values(), sp.keys()))
sorted_sp = sorted(zip(sp.values(), sp.keys()))  #  sp_list =list( zip(sp.values(), sp.keys()) ) 改写法只能消费一次

当进行这些计算时，请注意**zip创建了一个迭代器，它的内容只能被消费一次**。例如下面的代码就是错误的:  
```python
prices_and_names = zip(prices.values(),prices.keys()) 
print(min(prices_and_names)) #OK 
print(max(prices_and_names)) #ValueError:max() arg is an empty sequence
```

如果尝试在字典上执行常见的数据操作，将会发现它们只会处理键，而不是值。例如：  
`min(sp)` #Returns 'AAPL' ; `max(sp)` Returns 'IBM' 这很可能不是我们所期望的，因为实际上我们是尝试对字典的值做计算。 
可以利用**字典的values()方法**来解决这个问题：`min(sp.values())` #Returns 10.75;  `max(sp.values())` #Returns 612.78

## 1.9 在两个字典中寻找相同点
#### 问题：有两个字典，我们想找出它们中间可能相同的地方（相同的键、相同的值等）  ：keys()或items()方法执行常见的集合操作


In [16]:
ad = {'x':1,'y':2, 'z':3}; bd = {'w':10, 'x':11, 'y':2}

In [20]:
ad.keys() & bd.keys() # common

{'x', 'y'}

In [21]:
ad.keys() - bd.keys() # in ad not in bd

{'z'}

In [22]:
ad.items() & bd.items() # Find (key,value)pairs in common

{('y', 2)}

字典就是一系列键和值之间的映射集合。字典的keys()方法会返回keys-view对象，其中暴露了所有的键。  
关于`字典的键`有一个很少有人知道的**特性**，那就是它们也`支持常见的集合操作`，比如求并集、交集和差集。  
因此，如果需要对字典的键做常见的集合操作，那么就能直接使用keys-view对象而不必先将它们转化为集合。

字典的items()方法返回由(key,value)对组成的items-view对象。这个对象支持类似的集合操作，可用来完成找出两个字典间有哪些键值对有相同之处的操作。  
尽管类似，但**字典的values方法并不支持集合操作**。部分原因是因为在字典中键和值是不同的，从值的角度来看并不能保证所有的值都是唯一的。单这一条原因就使得某些特定的集合操作是有问题的。但是，如果必须执行这样的操作，还是可以先将值转化为集合来实现。

## 1.10 从序列中移除重复项且保持元素间顺序不变
如果序列中的值是可哈希(hashable)的(生存周期内必须是不可变的，eg:python的整数、浮点数、字符串、元组)，  
那么这个问题可以通过使用集合和生成器轻松解决。示例如下：


In [25]:
def del_dupe(items):
    seen =set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [26]:
dupelist = [1,5,2,1,9,1,5,10]
list(del_dupe(dupelist))

[1, 5, 2, 9, 10]

只有当序列中的元素是可哈希的时候才能这么做。如果想在不可哈希的对象（比如列表)序列中去除重复项，需要对上述代码稍作修改：
```python
def del_dupe(items, key = None)
...
for item in items: 
    val= item if key is None else key(item) 
    if val not in seen:
         yield item
        seen.add(val)
```

这里参数key的作用是指定一个**函数用来将序列中的元素转换为可哈希的类型**，这么做的目的是为了检测重复项。它可以像这样工作：
```python
a=[{'x':1,'y':2},{'x':1,'y':3},{'x':1,'y':2},{'x':2,'y':4}] 
list (dedupe(a,key=lambda d:(d['x'],d['y'])))  
# 结果[{'x':1,'y':2},{'x':1,y':3},{'x':2,y':4}]
```

set()可以去除重复项，但是这种方法不能保证元素间的顺序不变，因此得到的结果会被打乱。  前面展示的解决方案可避免出现这个问题。
本节中对生成器的使用反映出一个事实，那就是我们可能会希望这个函数尽可能的通用，不必绑定在只能对列表进行处理。  
比如，如果想读一个文件，去除其中重复的文本行，可以只需这样处理：
```
with open (somefile,'r')as f:   
        for line in dedupe(f):
```

## 1.11 对切片命名
问题： 我们的代码已经变得无法阅读，到处都是硬编码的切片索引，我们想将它们清理干净  
假设有一些代码用来从字符串的固定位置中取出具体的数据（比如从一个平面文件或类似的格式)   
##*## 0123456789012345678901234567890123456789012345678901234567890'   
record = '............100.......513.25....' 
 与其这样做，为什么不对切片命名呢？
SHARES =s1ice(20,32); PRICE slice(40,48)



作为一条基本准则，`代码中如果有很多硬编码的索引值，将导致可读性和可维护性都不佳`。  
例如，如果一年以后再回过头来看代码，你会发现自己很想知道当初编写这些代码时自己在想些什么。  
对切片命名的方法可以让我们对代码的功能有着更加清晰的认识。

一股来说，内置的slice()函数会创建一个切片对象，可以用在任何允许进行切片澡作的地方。如：
```python
items=[0,1,2,3,4,5,6] 
a=slice(2,4)   
items[2:4] #[2,3] 
items[a] #[2,3] 
items[a]=[10,11] #items [0,1,10,11,4,5,6] 
del items[a] #items [0,1,4,5,6]
```

## 1.12 找出序列中出现次数最多的元素
#### **collections模块中的Counter类**正是为此类问题所设计,`most_common()`方法可以查找序列中出现次数

eg:假设有一个列表，列表中是一系列的单词，我们想找出哪些单词出现的最为频繁:

In [8]:
words = ['look','into','my','eyes','look','into','my','eyes', 'the','eyes','the','eyes','the','eyes','not','around','the', 'eyes',"don't",'look','around','the','eyes','look','into', 'my','eyes',"you're",'under']

In [9]:
from collections import Counter 
wordscounts = Counter (words)   #Counter对象
topThreeWord = wordscounts.most_common(3) 
print(topThreeWord)

[('eyes', 8), ('the', 5), ('look', 4)]


可以给Counter对象**提供任何可哈希的对象序列**作为输入。在底层实现中，Counter是一个字典，  
在元素和它们出现的次数间做了映射。例如：
`wordscounts['not']` #1
不用说，当面对任何需要对数据制表或计数的问题时，Counter对象都是你手边的得力工具。  
比起利用字典自己手写算法，更应该采用这种方式完成任务。#注：Counter对象可以做数学运算操作

## 1.13 通过公共键对字典列表排序
#### 一个字典列表，根据一个或多个字典中的值来对列表排序 (列表中的值是一个个字典)
利用**operator模块中的itemgetter**函数对这类结构进行排序

In [10]:
dlis = [{'fname':'Brian','lname':'Jones','uid':1003}, 
{'fname':'David','lname':'Beazley','uid':1002},{'fname':'John','lname':'Cleese','uid':1001}]

In [12]:
from operator import itemgetter
dlis_by_fname = sorted(dlis, key = itemgetter('fname')) #将列表中的多个字典按字典值fname排序
dlis_by_uid = sorted(dlis, key = itemgetter('uid', 'lname')) #itemgetter可接受多个参

在这个例子中，dlis被传递给内建的sorted函数，该函数接受一个关键字参数key  
这个参数应该代表一个可调用对象（callable),该对象从dlis中接受一个单独的元素作为输入并返回一个用来做排序依据的值
itemgetter()函数创建的就是这样一个可调用对象

函数operator.itemgetter()接受的参数可作为查询的标记，用来从dlis的记录中提取出所需要的值。  
它可以是字典的键名称、用数字表示的列表元素或是任何可以传给对象的__getitem__()方法的值。  
如果传多个标记给itemgetter(),那么它产生的可调用对象将返回一个包含所有元素在内的元组，然后sorted()将根据对元组的排序结果来排列输出结果。  
如果想同时针对多个字段做排序（比如例子中的姓和名)，那么这是非常有用的。 注：max和min函数同样适用

## 1.14 对不原生支持比较操作的对象排序
#### 想在同一个类的实例之间做排序，但是它们并不原生支持比较操作

内建的sorted()函数可接受一个用来传递可调用对象(callable)的参数key,  
而该可调用对象会返回待排序对象中的某些值，sorted则利用这些值来比较对象。  
例如，如果应用中有一系列的User对象实例，而我们想通过userid属性来对它们排序，则可以提供一个可调用对象将User实例作为输入然后返回user_id

In [ ]:
class User:
    def __init__(self, user_id) -> None:    
        self.user_id = user_id
    def __repr__(self) -> str:  #自定义类中实现 “自我描述” 的功能，必须重写 repr 方法
        return 'User({})'.format(self.user_id)
        
    